In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')

Mounted at /content/drive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def read_file_as_one_document(filename):
    with open(filename, 'r') as f:
        text = f.read()
        tokens = word_tokenize(text)
        return tokens

def get_documents_from_subfolders(base_path, subfolder, num_files=2, subset = False):
    documents = []
    # labels = []
    subfolder_path = os.path.join(base_path, subfolder)
    print(f"subfolder_path: {subfolder_path}")

    # if subset:
    #   files = sorted(os.listdir(subfolder_path))[:num_files]  # Get the first x files from each subfolder
    # else:
    #   files = sorted(os.listdir(subfolder_path))              # Get all the files
    files = os.listdir(subfolder_path)
    for filename in files:
        if filename.endswith('.txt'):
            file_path = os.path.join(subfolder_path, filename)
            documents.append(read_file_as_one_document(file_path))
            # labels.append(subfolder)  # Add the label for the subfolder
    return documents

# document vector
def document_vector(word2vec_model, doc_tokens):
    doc_tokens = [token for token in doc_tokens if token in word2vec_model.wv.key_to_index]
    if not doc_tokens:
        return np.zeros(word2vec_model.vector_size)
    return np.mean(word2vec_model.wv[doc_tokens], axis=0)




# base_path = '/content/drive/MyDrive/malware2' #tiffany
base_path = '/content/drive/MyDrive/REU/ResearchProject/v077_clean/' #kylie
# subfolders = ['zbot']
# subfolders = ['Vobfus', 'Zbot', 'Diplugem', 'Obfuscator', 'Vundo',
#               'VBInject', 'Delf', 'Beebone', 'Winwebsec', 'Enterak',
#               'OnLineGames', 'Startpage', 'Allaple', 'Injector', 'Systex.A',
#               'Expiro.BK', 'FakeRean', 'Small', 'Toga!rfn', 'Lamech.B']
#### from maleware2
# subfolders = ['winwebsec', 'zeroaccess'] #'zbot',
#### from v077
subfolders = ['Vobfus', 'Diplugem', 'Obfuscator', 'Vundo', 'VBInject',
              'Delf', 'Beebone', 'Enterak.A','OnLineGames',
              'Startpage', 'Allaple.A', 'Injector', 'Systex.A', 'Expiro.BK',
              'FakeRean', 'Small', 'Toga!rfn', 'Lamechi.B', 'CeeInject',
              'Renos', 'Hotbar', 'DelfInject']

for family in subfolders:
  # Get the documents from each subfolder
  documents = get_documents_from_subfolders(base_path, subfolder = family)

  print(f"family: {family}")
  print(f"documents: {len(documents)}")
  print("------------------------------------------------------")
  # print(f"labels: {len(labels)}")

  #Word2Vec model
  word2vec_model = Word2Vec(sentences=documents, vector_size=104,  window=10, min_count=0, workers=4, seed=42)

  #document embedding
  doc_vectors = [document_vector(word2vec_model, doc) for doc in documents]
  doc_vectors_df = pd.DataFrame(doc_vectors)

  #normalize to [0,1]
  max_val = doc_vectors_df.to_numpy().max()
  min_val = doc_vectors_df.to_numpy().min()
  range_vals = max_val - min_val

  for i in range(104):
    doc_vectors_df[i] = (doc_vectors_df[i] - min_val) / range_vals


  #save csv
  new_filename = str( "/content/drive/MyDrive/REU/ResearchProject/Diffusion_Opcodes/Fake_Malware_Generator/data/embeddings/top25_104/" + family + ".csv" )
  doc_vectors_df.to_csv(new_filename, header=False, index=False)


subfolder_path: /content/drive/MyDrive/REU/ResearchProject/v077_clean/Vobfus


#

In [ ]:
from google.colab import files

# !mkdir -p "/content/drive/My Drive/Malware_Embeddings"

# export the csv without indexes
## CHANGE FAMILY NAME WHEN SAVING

# doc_vectors_df.to_csv('/content/drive/My Drive/Malware_Embeddings/doc_vectors.csv', header=False, index=False) #tiffany
doc_vectors_df.to_csv('/content/drive/MyDrive/REU/ResearchProject/Diffusion_Opcodes/Fake_Malware_Generator/data/embeddings/winwebsec.csv', header=False, index=False)


# download the csv
# files.download('/content/drive/My Drive/Malware_Embeddings/doc_vectors.csv')


# Directory Counts

In [ ]:
import os

def count_files_in_directory(directory):
    file_counts = {}
    for root, dirs, files in os.walk(directory):
        for dir in dirs:
            dir_path = os.path.join(root, dir)
            num_files = sum([len(files) for _, _, files in os.walk(dir_path)])
            file_counts[dir_path] = num_files
    return file_counts


dir = '/content/drive/MyDrive/REU/ResearchProject/v077_clean'
file_counts = count_files_in_directory(dir)
sorted_file_counts = sorted(file_counts.items(), key=lambda item: item[1], reverse=True)
for dir_path, count in sorted_file_counts:
    print(f"{dir_path}: {count} files")

/content/drive/MyDrive/REU/ResearchProject/v077_clean/Vobfus: 4204 files
/content/drive/MyDrive/REU/ResearchProject/v077_clean/Zbot: 2353 files
/content/drive/MyDrive/REU/ResearchProject/v077_clean/Diplugem: 2269 files
/content/drive/MyDrive/REU/ResearchProject/v077_clean/Obfuscator: 2102 files
/content/drive/MyDrive/REU/ResearchProject/v077_clean/Vundo: 1877 files
/content/drive/MyDrive/REU/ResearchProject/v077_clean/VBInject: 1688 files
/content/drive/MyDrive/REU/ResearchProject/v077_clean/Delf: 1679 files
/content/drive/MyDrive/REU/ResearchProject/v077_clean/Beebone: 1629 files
/content/drive/MyDrive/REU/ResearchProject/v077_clean/Winwebsec: 1625 files
/content/drive/MyDrive/REU/ResearchProject/v077_clean/Enterak.A: 1530 files
/content/drive/MyDrive/REU/ResearchProject/v077_clean/OnLineGames: 1366 files
/content/drive/MyDrive/REU/ResearchProject/v077_clean/Startpage: 1313 files
/content/drive/MyDrive/REU/ResearchProject/v077_clean/Allaple.A: 1294 files
/content/drive/MyDrive/REU/Res